# 데이터 불러오기

In [ ]:
import os
import shutil
import yaml
from ultralytics import YOLO


In [ ]:
# 필터링된 데이터셋 저장 경로
filtered_dataset = './ppe_filtered'
new_yaml = os.path.join(filtered_dataset, 'data.yaml')

# 모델 학습

In [ ]:

model = YOLO('yolov11n.pt')

results = model.train(
    data=new_yaml,
    epochs=100,
    imgsz=640,
    batch=16,
    workers=2,
    name='ppe_detection',
    patience=10,
    save=True,
    plots=True
)


# 모델 성능 평가

In [ ]:
model = YOLO('runs/detect/ppe_detection2/weights/best.pt')

metrics = model.val(data=new_yaml)

print(f'mAP50: {metrics.box.map50:.3f}')
print(f'mAP50-95: {metrics.box.map:.3f}')


# 테스트

In [ ]:
test_img_dir = os.path.join(filtered_dataset, 'test/images')

results = model.predict(
    source=test_img_dir,
    save=True,
    conf=0.25,
    project='runs/detect',
    name='test_results',
    exist_ok=True
)

print(f'테스트 결과 저장 위치: runs/detect/test_results')


In [ ]:
import glob
import matplotlib.pyplot as plt

# 테스트 이미지 불러오기
test_images = glob.glob(os.path.join(test_img_dir, '*.jpg'))[:12]

# 그리드 설정
n_cols = 4
n_rows = (len(test_images) + n_cols - 1) // n_cols

# Figure 생성
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
axes = axes.flatten()

# 예측 및 표시
for idx, img_path in enumerate(test_images):
    results = model(img_path)
    img_rgb = results[0].plot()[:, :, ::-1]  # BGR -> RGB
    
    axes[idx].imshow(img_rgb)
    axes[idx].axis('off')

# 빈 칸 제거
for idx in range(len(test_images), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()
